# House Pricing Prediction in Tangerang

<b>Import Libraries</b>

In [37]:
import requests
from contextlib import closing
import csv

<b>Baca file "harga_rumah.txt"</b> 

In [38]:
# STEP 1: 
# Baca file "harga_rumah.txt"
url = "https://storage.googleapis.com/dqlab-dataset/harga_rumah.txt"
data_harga_rumah = []
with closing(requests.get(url, stream=True)) as r:
    f = (line.decode('utf-8') for line in r.iter_lines())
    data_harga_rumah = [row for row in csv.reader(f)]

In [39]:
data_harga_rumah

[['tanah', 'bangunan', 'jarak_ke_pusat', 'harga'],
 ['70', '50', '15', '500'],
 ['70', '60', '30', '400'],
 ['70', '60', '55', '300'],
 ['100', '50', '30', '700'],
 ['100', '70', '25', '1000'],
 ['100', '70', '50', '650'],
 ['120', '100', '20', '2000'],
 ['120', '80', '50', '1200'],
 ['150', '100', '50', '1800'],
 ['150', '90', '15', '3000']]

* Buat list of dict dengan nama harga rumah

In [40]:
key_harga_rumah = data_harga_rumah[0]
harga_rumah = []
for baris_harga_rumah in data_harga_rumah[1:]:
    dict_harga_rumah = dict()
    for i in range(len(baris_harga_rumah)):
        dict_harga_rumah[key_harga_rumah[i]] = baris_harga_rumah[i]
    harga_rumah.append(dict_harga_rumah)
print(harga_rumah)

[{'tanah': '70', 'bangunan': '50', 'jarak_ke_pusat': '15', 'harga': '500'}, {'tanah': '70', 'bangunan': '60', 'jarak_ke_pusat': '30', 'harga': '400'}, {'tanah': '70', 'bangunan': '60', 'jarak_ke_pusat': '55', 'harga': '300'}, {'tanah': '100', 'bangunan': '50', 'jarak_ke_pusat': '30', 'harga': '700'}, {'tanah': '100', 'bangunan': '70', 'jarak_ke_pusat': '25', 'harga': '1000'}, {'tanah': '100', 'bangunan': '70', 'jarak_ke_pusat': '50', 'harga': '650'}, {'tanah': '120', 'bangunan': '100', 'jarak_ke_pusat': '20', 'harga': '2000'}, {'tanah': '120', 'bangunan': '80', 'jarak_ke_pusat': '50', 'harga': '1200'}, {'tanah': '150', 'bangunan': '100', 'jarak_ke_pusat': '50', 'harga': '1800'}, {'tanah': '150', 'bangunan': '90', 'jarak_ke_pusat': '15', 'harga': '3000'}]


* Tranform list of dictionary to Dataframe with Pandas

In [41]:
import pandas as pd
df = pd.DataFrame(harga_rumah, columns= ['tanah','bangunan','jarak_ke_pusat','harga'])
df

,tanah,bangunan,jarak_ke_pusat,harga
0,70,50,15,500
1,70,60,30,400
2,70,60,55,300
3,100,50,30,700
4,100,70,25,1000
5,100,70,50,650
6,120,100,20,2000
7,120,80,50,1200
8,150,100,50,1800
9,150,90,15,3000


In [42]:
# STEP 2:
# Buat fungsi  get_all_specified_attribute yang menerima parameter list_of_dictionary 
# (tipe data list yang berisikan sekumpulan tipe data dictionary) dan specified_key 
# (tipe data string). Fungsi akan mengembalikan sebuah list yang berisikan seluruh 
# atribut dengan kunci (key) specified_key. 
def get_all_specified_attributes(list_of_dictionary, specified_key):
    list_attributes = []
    for data in list_of_dictionary:
        attribute = data[specified_key]
        list_attributes.append(attribute)
    return list_attributes

In [43]:
# STEP 3: 
# Buat fungsi fungsi min_value yang menerima parameter list_attributes (berupa 
# tipe data list) dan mengembalikan nilai terkecil dalam list_attributes 
def min_value(list_attributes):
    min_attribute = 9999
    for attr in list_attributes:
        if int(attr) < min_attribute:
            min_attribute = int(attr)
    return min_attribute
# Buat fungsi dan max_value yang menerima parameter list_attribute dan 
# mengembalikan nilai terbesar dalam list_attributes.	
def max_value(list_attributes):
    max_attribute = -9999
    for attr in list_attributes:
        if int(attr) > max_attribute:
            max_attribute = int(attr)
    return max_attribute

In [44]:
# STEP 4: 
# Buat fungsi transform_attribute yang menerima parameter attr (sebuah 
# bilangan), max_attr (sebuah bilangan) dan min_attr (sebuah bilangan) 
# yang mengembalikan nilai transformasi dari sebuah attribute.
def transform_attribute(attr, max_attr, min_attr):
    nilai_transformasi = (attr - min_attr) / (max_attr - min_attr)
    return nilai_transformasi

In [45]:
# STEP 5:
# Buat fungsi data_transformation yang menerima parameter list_of_dictionary 
# (sebuah list yang berisikan tipe data dictionary) dan list_attribute_names 
# (sebuah list yang berisikan tipe data string) mengembalikan hasil 
# transformasi data dari list_of_dictionary berdasarkan list_attribute_names 
# dan attr_info telah dispesifikasikan.
def data_transformation(list_of_dictionary, list_attribute_names):
    attr_info = {}
    for attr_name in list_attribute_names:
        specified_attributes = get_all_specified_attributes(list_of_dictionary, attr_name)
        max_attr = max_value(specified_attributes)
        min_attr = min_value(specified_attributes)
        attr_info[attr_name] = {'max': max_attr, 'min': min_attr}
        data_idx = 0
        while(data_idx < len(list_of_dictionary)):
            list_of_dictionary[data_idx][attr_name] = transform_attribute(int(list_of_dictionary[data_idx][attr_name]), max_attr, min_attr)
            data_idx += 1
    return list_of_dictionary, attr_info

In [46]:
# STEP 6:
# Berdasarkan data baru dan attr_info ini, buat fungsi transform_data yang
# menerima parameter data dan attr_info dan mengembalikan nilai atribut 
# dari data baru yang telah ditransformasikan.
def transform_data(data, attr_info):
    for key_name in data.keys():
        data[key_name] = (data[key_name] - attr_info[key_name]['min']) / (
                          attr_info[key_name]['max'] - attr_info[key_name]['min'])
    return data

In [47]:
# STEP 7:
# Buat fungsi yang digunakan untuk sistem prediksi harga berdasarkan 
# nilai kemiripan atribut, yaitu argument input data dan list_of_data!
def abs_value(value):
    if value < 0:
        return - value
    else:
        return value
def price_based_on_similarity(data, list_of_data):
    prediksi_harga = 0
    perbedaan_terkecil = 999
    for data_point in list_of_data:
        perbedaan= abs_value(data['tanah'] - data_point['tanah'])
        perbedaan+= abs_value(data['bangunan'] - data_point['bangunan'])
        perbedaan+= abs_value(data['jarak_ke_pusat'] - data_point['jarak_ke_pusat'])
        if perbedaan < perbedaan_terkecil:
            prediksi_harga = data_point['harga']
            perbedaan_terkecil = perbedaan
    return prediksi_harga

In [48]:
# STEP 7:
# Buat fungsi yang digunakan untuk sistem prediksi harga berdasarkan 
# nilai kemiripan atribut, yaitu argument input data dan list_of_data!
def abs_value(value):
    if value < 0:
        return - value
    else:
        return value
def price_based_on_similarity(data, list_of_data):
    prediksi_harga = 0
    perbedaan_terkecil = 999
    for data_point in list_of_data:
        perbedaan= abs_value(data['tanah'] - data_point['tanah'])
        perbedaan+= abs_value(data['bangunan'] - data_point['bangunan'])
        perbedaan+= abs_value(data['jarak_ke_pusat'] - data_point['jarak_ke_pusat'])
        if perbedaan < perbedaan_terkecil:
            prediksi_harga = data_point['harga']
            perbedaan_terkecil = perbedaan
    return prediksi_harga

In [49]:
# STEP 8:
# Hitung harga rumah yang telah ditransformasikan ke dalam variabel 
# harga_rumah berikut dengan atributnya attr_info
harga_rumah, attr_info = data_transformation(harga_rumah,
                                            ['tanah','bangunan','jarak_ke_pusat'])
# Gunakan variabel data untuk memprediksi harga rumah
data = {'tanah': 110, 'bangunan': 80, 'jarak_ke_pusat': 35}
# Transformasikan data tersebut dengan dengan menggunakan attr_info yang telah 
# diperoleh yang kembali disimpan ke variabel data.
data = transform_data(data, attr_info)
# Hitunglah prediksi harga dari variabel data tersebut.
harga = price_based_on_similarity(data, harga_rumah)
print("Prediksi harga rumah: ", harga)

Prediksi harga rumah:  1200
